### East Coastal Plain Open Pine Birds

This is a draft indicator for the 202 base blueprint.

It is the same as the 2022 indicator, except zero values were added to represent the exent of the modeling effort.

Created by Amy Keister, last run by Amy Keister on 1 June 2023. It took 10 minutes run.

In [16]:
import os
import arcpy

In [17]:
import time
start = time.time()

In [18]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\EastCoastalPlainO\EastCoastalPlainOpe.gdb"

In [19]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\EastCoastalPlainO\EastCoastalPlainOpenPineBirds.tif"

In [20]:
# define sub-indicator outputs

In [21]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [22]:
# define inputs
OP= r"F:\GIS_DATA\DecisionSupportTools\EGCPJV\bird_priority\bird_priority.tif"
FP= r"F:\GIS_DATA\WaterResources\Estimated_floodplain_CONUS\Estimated_floodplain_CONUS.tif"
NLCD= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"

## Start Analysis

In [23]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [24]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\EastCoastalPlainO\EastCoastalPlainOpe.gdb


## bring in EGCP Open Pine Decision Support Tool

In [10]:
# Bring in the EGCP tif
arcpy.conversion.RasterToGeodatabase(OP, OutWorkspace, '')

<Result ''>

In [11]:
# remove EPA estimated floodplains, to try to narrow down the results to potential open pine habitat
with arcpy.EnvManager(outputCoordinateSystem=NLCD, extent= OP, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.Con(FP, "bird_priority", None, "Value = 0"); out_raster.save("bird_priority_noFP")

In [12]:
# pull out evergreen and mixed forest (42,43) from NLCD
with arcpy.EnvManager(outputCoordinateSystem=NLCD, extent= OP, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.Con(NLCD, 1, 0, "Value = 42 Or Value = 43"); out_raster.save("pine")

In [13]:
# remove areas that are not either mixed or evergreen forest from the 2019 NLCD, to try to narrow down the results to current open pine habitat
with arcpy.EnvManager(outputCoordinateSystem=NLCD, extent= OP, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.Times("pine", "bird_priority_noFP"); out_raster.save("bird_priority_noFP_pine")

In [14]:
# reclassify to bin

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
out_raster = arcpy.sa.Reclassify("bird_priority_noFP_pine", "VALUE", "0 0 0;1 20 1;20 40 2;40 60 3;60 80 4;80 100 5", "DATA"); out_raster.save("rclss")

In [15]:
# get full extent of original data
with arcpy.EnvManager(outputCoordinateSystem=NLCD, extent= OP, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.Con("bird_priority", 0, 0, "Value > -1 "); out_raster.save("full0")

In [37]:
# add in zeros
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=OP, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.CellStatistics("full0;rclss", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [38]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [39]:
# export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [40]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 5:
        return '5 = High priority for open pine conservation for focal bird species (Bachman’s sparrow, red-cockaded woodpecker, Henslow’s sparrow, red-headed woodpecker, Northern bobwhite, and brown-headed nuthatch) (score >80-100)'
    elif value == 4:
        return '4 = Medium-high priority (score >60-80)'
    elif value == 3:
        return '3 = Medium priority (score >40-60)'
    elif value == 2:
        return '2 = Medium-low priority (score >20-40)'
    elif value == 1:
        return '1 = Low priority (score 0-20)'
    elif value == 0:
        return '0 = Not a priority (not identified as upland pine)'
"""

In [41]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\EastCoastalPlainO\\EastCoastalPlainOpenPineBirds.tif'>

In [42]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 103
	if Value == 4:
		return 189
	if Value == 3:
		return 228
	if Value == 2:
		return 202
	if Value == 1:
		return 226
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 0
	if Value == 4:
		return 12
	if Value == 3:
		return 63
	if Value == 2:
		return 150
	if Value == 1:
		return 214
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 31
	if Value == 4:
		return 80
	if Value == 3:
		return 152
	if Value == 2:
		return 200
	if Value == 1:
		return 233
	else:
		return 255
		
"""

In [43]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\EastCoastalPlainO\\EastCoastalPlainOpenPineBirds.tif'>

In [23]:
end = time.time()
print(end - start)

997.4286813735962
